find one region's neighboring regions:
1. input data: geometry information of Manhattan
2. for loop each region in above data, and use intersects(shapely) to determine whether two regions intersect
3. store target region id and its neighboring region isd. just like that {'4': ['79', '148', '224', '232'], '24': ['41', '43', '151', '166'],...}

In [38]:
import json
import geopandas as gpd

In [39]:
#input data contains region is and its corresponding geometry information
input_region_geo_path = './data_set/NYC_geo/NYC_Taxi_Zones.geojson'
input_region_geo_Manhanttan_path = './data_set/NYC_geo/NYC_Taxi_Zones_Manhattan.geojson'
#store results of neighboring regions
#neighbor_region_optimization_path = './data_set/NYC_geo/neighbor_region.json'
neighbor_region_optimization_path = './data_set/NYC_geo/neighbor_region_for_optimization.json'

In [23]:
region_gpd = gpd.read_file(input_region_geo_path)
region_gpd = region_gpd[region_gpd["borough"] == "Manhattan"]
region_gpd = region_gpd.loc[:, ['location_id', 'borough', 'geometry']]
region_gpd = region_gpd.rename(columns={'location_id': 'region_id'})
region_gpd = region_gpd.reset_index() 
region_gpd.count()

index        63
region_id    63
borough      63
geometry     63
dtype: int64

In [29]:
region_gpd

,index,region_id,borough,geometry
0,3,4,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
1,9,24,Manhattan,"MULTIPOLYGON (((-73.95954 40.79872, -73.96004 ..."
2,12,12,Manhattan,"MULTIPOLYGON (((-74.01566 40.70483, -74.01540 ..."
3,13,13,Manhattan,"MULTIPOLYGON (((-74.01244 40.71906, -74.01282 ..."
5,41,45,Manhattan,"MULTIPOLYGON (((-73.99750 40.71407, -73.99744 ..."
7,44,43,Manhattan,"MULTIPOLYGON (((-73.97255 40.76490, -73.97301 ..."
8,47,48,Manhattan,"MULTIPOLYGON (((-73.99118 40.75533, -73.99163 ..."
9,49,50,Manhattan,"MULTIPOLYGON (((-73.99668 40.77351, -73.99434 ..."
10,68,68,Manhattan,"MULTIPOLYGON (((-74.00202 40.74044, -74.00252 ..."
11,73,79,Manhattan,"MULTIPOLYGON (((-73.98378 40.72154, -73.98382 ..."


In [40]:
def correspond_regionid_to_regiongeo(input_region_geo_path, input_region_geo_Manhanttan_path):

    # get region multiploygon
    region_gpd = gpd.read_file(input_region_geo_path)
    region_gpd = region_gpd[region_gpd["borough"] == "Manhattan"]

    region_gpd = region_gpd.loc[:, ['location_id', 'borough', 'geometry']]
    region_gpd = region_gpd.rename(columns={'location_id': 'region_id'})
    region_gpd = region_gpd.reset_index(drop=True)
    
    region_gpd = region_gpd[~region_gpd["region_id"].isin(['120', '128', '244', '243', '116', '152', '42', '166', '41', '74', '127'])]
    
    #with open(input_region_geo_Manhanttan_path, 'w') as f:
        #f.write(region_gpd.to_json())

    # sjoin region id and tmc code by geometry
    
    neighbor_region = {}
    
    for _i in region_gpd.index:
        
        target_region = region_gpd.loc[_i]
        
        target_region_id = target_region['region_id']
        target_polygon = target_region['geometry']

        for _j in region_gpd.index:
            if _j != _i:
                other_region = region_gpd.loc[_j]
                other_region_id = other_region['region_id']
                other_polygon = other_region['geometry']
                
                #if target_polygon.touches(other_polygon):
                if target_polygon.intersects(other_polygon):
                
                    if target_region_id not in neighbor_region.keys():
                        
                        neighbor_region[target_region_id] = []
                    neighbor_region[target_region_id].append(other_region_id)
    print('=========neighbor_region===========')
    print(neighbor_region)

    out_file = open(neighbor_region_optimization_path, "w")
    json.dump(neighbor_region, out_file)


In [41]:
if __name__ == '__main__':
    correspond_regionid_to_regiongeo(input_region_geo_path, input_region_geo_Manhanttan_path)

=========neighbor_region===========
{'4': ['79', '148', '224', '232'], '24': ['43', '151'], '12': ['13', '88', '261'], '13': ['12', '231', '261'], '45': ['144', '148', '209', '231', '232'], '43': ['24', '75', '151', '142', '163', '239', '236', '237', '238'], '48': ['50', '68', '100', '142', '143', '163', '230', '246'], '50': ['48', '142', '143', '246'], '68': ['48', '90', '100', '158', '186', '246', '249'], '79': ['4', '107', '113', '114', '148', '224', '232', '234'], '75': ['43', '236', '263', '262'], '87': ['88', '209', '261'], '88': ['12', '87', '261'], '90': ['68', '113', '186', '234', '249'], '125': ['114', '158', '211', '231', '249'], '100': ['48', '68', '161', '164', '186', '230'], '107': ['79', '113', '137', '164', '170', '224', '234'], '113': ['79', '90', '107', '114', '234', '249'], '114': ['79', '125', '113', '144', '211', '249'], '151': ['24', '43', '238'], '140': ['141', '229', '263', '262'], '137': ['107', '170', '224', '233'], '141': ['140', '162', '229', '236', '237', '